In [2]:
# necessary imports
import numpy as np, sys
from keras.datasets import mnist

np.random.seed(1)

In [3]:
# load and prepare data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

images, labels = (x_train[0:1000].reshape(1000, 28*28) / 255, y_train[0:1000])

test_images = x_test.reshape(len(x_test), 28*28) / 255
test_labels = np.zeros((len(y_test), 10))

In [4]:
# code our test data
one_hot_labels = np.zeros((len(labels), 10))

for i, l in enumerate(labels):
    one_hot_labels[i][l] = 1

labels = one_hot_labels

In [5]:
# code our train data
for i, l in enumerate(y_test):
    test_labels[i][l] = 1

In [6]:
# add activation functions
def tanh(x):
    return np.tanh(x)

def tanh2deriv(output):
    return 1 - (output ** 2)

def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

In [7]:
# setup network parameters
alpha, iterations = (2, 300)
pixels_perimage, num_labels = (784, 10)
batch_size = 128

input_rows = 28
input_cols = 28

# setup convulation kernel parameters
kernel_rows = 3
kernel_cols = 3
num_kernels = 16

# setup hidden layer parameters
hidden_size = ((input_rows - kernel_rows) * (input_cols - kernel_cols)) * num_kernels

# setup weights
# setup weights for colnvulation layer
kernels = 0.02 * np.random.random((kernel_rows * kernel_cols, num_kernels)) - 0.01
# setup hidden layer weights
weights_1_2 = 0.2 * np.random.random((hidden_size, num_labels)) - 0.1

In [8]:
# define function for getting image sections
def get_image_section(layer, row_from, row_to, col_from, col_to):
    section = layer[:, row_from:row_to, col_from:col_to]
    return section.reshape(-1, 1, row_to - row_from, col_to - col_from)

In [9]:
%%time

# write down train section for our model
for j in range(iterations):
    correct_cnt = 0
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end=((i * batch_size),((i+1)*batch_size))
        layer_0 = images[batch_start:batch_end]
        layer_0 = layer_0.reshape(layer_0.shape[0], 28, 28)

        # split on kernels
        sects = list()
        for row_start in range(layer_0.shape[1] - kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,
                                         row_start,
                                         row_start + kernel_rows,
                                         col_start,
                                         col_start + kernel_cols)
                sects.append(sect)
        # do some tech reshaping
        expanded_input = np.concatenate(sects, axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0] * es[1], -1)
        # fit our model
        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0], -1))
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = softmax(np.dot(layer_1, weights_1_2))
        # counting correct answers on train data
        for k in range(batch_size):
            labelset = labels[k + batch_size:batch_size + k + 1]
            _inc = int(np.argmax(layer_2[k:k + 1]) == np.argmax(labelset))
            correct_cnt += _inc
        # calc gradinent(simplyfied backprob)
        layer_2_delta = (labels[batch_start:batch_end] - layer_2)\
                        / (batch_size * layer_2.shape[0])
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * tanh2deriv(layer_1)
        layer_1_delta *= dropout_mask
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        l1d_reshape = layer_1_delta.reshape(kernel_output.shape)
        k_update = flattened_input.T.dot(l1d_reshape)
        kernels -= alpha * k_update
    # count correct answers on test
    test_correct_cnt = 0

    for i in range(len(test_images)):
        layer_0 = test_images[i:i + 1]
        layer_0 = layer_0.reshape(layer_0.shape[0], 28, 28)

        sects = list()
        for row_start in range(layer_0.shape[1] - kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,
                                         row_start,
                                         row_start + kernel_rows,
                                         col_start,
                                         col_start + kernel_cols)
                sects.append(sect)

        expanded_input = np.concatenate(sects, axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0] * es[1], -1)
    
        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0], -1))
        layer_2 = np.dot(layer_1, weights_1_2)

        test_correct_cnt += int(np.argmax(layer_2) ==
                                np.argmax(test_labels[i:i + 1]))
    if(j % 1 == 0):
        sys.stdout.write("\n"+ \
         "I:" + str(j) + \
         " Test-Acc:"+str(test_correct_cnt/float(len(test_images)))+\
         " Train-Acc:" + str(correct_cnt/float(len(images))))


I:0 Test-Acc:0.0288 Train-Acc:0.078
I:1 Test-Acc:0.0273 Train-Acc:0.072
I:2 Test-Acc:0.028 Train-Acc:0.07
I:3 Test-Acc:0.0292 Train-Acc:0.08
I:4 Test-Acc:0.0339 Train-Acc:0.082
I:5 Test-Acc:0.0478 Train-Acc:0.097
I:6 Test-Acc:0.076 Train-Acc:0.093
I:7 Test-Acc:0.1316 Train-Acc:0.102
I:8 Test-Acc:0.2137 Train-Acc:0.092
I:9 Test-Acc:0.2941 Train-Acc:0.092
I:10 Test-Acc:0.3563 Train-Acc:0.115
I:11 Test-Acc:0.4023 Train-Acc:0.114
I:12 Test-Acc:0.4358 Train-Acc:0.113
I:13 Test-Acc:0.4473 Train-Acc:0.126
I:14 Test-Acc:0.4389 Train-Acc:0.115
I:15 Test-Acc:0.3951 Train-Acc:0.13
I:16 Test-Acc:0.2222 Train-Acc:0.13
I:17 Test-Acc:0.0613 Train-Acc:0.097
I:18 Test-Acc:0.0266 Train-Acc:0.103
I:19 Test-Acc:0.0127 Train-Acc:0.077
I:20 Test-Acc:0.0133 Train-Acc:0.07
I:21 Test-Acc:0.0185 Train-Acc:0.081
I:22 Test-Acc:0.0363 Train-Acc:0.077
I:23 Test-Acc:0.0928 Train-Acc:0.075
I:24 Test-Acc:0.1994 Train-Acc:0.084
I:25 Test-Acc:0.3086 Train-Acc:0.11
I:26 Test-Acc:0.4276 Train-Acc:0.122
I:27 Test-Acc:0.53